In [ ]:
import pickle
import numpy  as np

In [ ]:
import pickle
with open("train_qa-220120-145526.txt","rb") as fp:
    train_data = pickle.load(fp)

FileNotFoundError: ignored

In [ ]:
train_data
print(len(train_data))

In [ ]:
with open("test_qa-220120-145430.txt","rb") as fp:
    test_data = pickle.load(fp)

In [ ]:
test_data
print(len(test_data))

In [ ]:

train_data[0]

In [ ]:
' '.join(train_data[0][0])

In [ ]:
' '.join(train_data[0][1])

In [ ]:
train_data[0][2]

In [ ]:
#Set up vocublary
vocab = set()

In [ ]:
all_data = test_data+train_data

In [ ]:
type(all_data)

In [ ]:
for story, question, answer in all_data:
    vocab = vocab.union(set(story))
    vocab = vocab.union(set(question))

In [ ]:
vocab.add('yes')
vocab.add('no')

In [ ]:
len(vocab)

In [ ]:
vocab_len = len(vocab)+1

In [ ]:
for data in all_data:
    print(data[0])
    print(len(data[0]))
    print('\n')

In [ ]:
max_story_len = max([len(data[0]) for data in all_data ])
max_story_len

In [ ]:

max_question_len = max([len(data[1]) for data in all_data ])
max_question_len

In [ ]:
#vectorize(convert dataset into mathematical form)
vocab

In [ ]:
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer = Tokenizer(filters = [])

In [ ]:
tokenizer.fit_on_texts(vocab)

In [ ]:
tokenizer.word_index

In [ ]:
train_story_text = []
train_question_text = []
train_answers = []

for story, question, answer in train_data:
    train_story_text.append(story)
    train_question_text.append(question)

In [ ]:
train_story_seq = tokenizer.texts_to_sequences(train_story_text)

In [ ]:
len(train_story_text)

In [ ]:
len(train_story_seq)

In [ ]:
train_story_seq

In [ ]:
train_story_text

In [ ]:
def vectorize_stories(data, word_index = tokenizer.word_index,
    max_story_len = max_story_len, max_question_len = max_question_len):
    X = [] #stories
    Xq = [] #query/question
    Y = [] #answer

    for story, query, answer in data:
        x = [word_index[word.lower()] for word in story]
        xq = [word_index[word.lower()] for word in query]
        y = np.zeros(len(word_index)+1)
        y[word_index[answer]] = 1

        X.append(x)
        Xq.append(xq)
        Y.append(y)
    
        return(pad_sequences(X, maxlen = max_story_len),
           pad_sequences(Xq, maxlen = max_question_len),
           np.array(Y) )

In [ ]:
inputs_train, queries_train, answer_train = vectorize_stories(train_data)
inputs_test, queries_test, answer_test = vectorize_stories(test_data)

In [ ]:
inputs_train

In [ ]:
queries_test

In [ ]:
answer_test

In [ ]:
tokenizer.word_index['yes']

In [ ]:
tokenizer.word_index['no']

In [ ]:
from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout, add, Dot, concatenate, LSTM

In [ ]:
input_sequence = Input((max_story_len,))
question = Input((max_question_len,))

In [ ]:
#Input Encoder m
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim = vocab_len, output_dim = 64))
input_encoder_m.add(Dropout(0.3))

In [ ]:
#Input Encoder c
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim = vocab_len, output_dim = max_question_len))
input_encoder_c.add(Dropout(0.3))

In [ ]:
#Questions Encoder
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim = vocab_len, output_dim = 64, input_length = max_question_len))
question_encoder.add(Dropout(0.3))

In [ ]:
#Encode the sequences
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

In [ ]:
match = tf.keras.layers.Dot(axes=(2, 2))([input_encoded_m, question_encoded])
match = Activation('softmax')(match)

In [ ]:
response = add([match , input_encoded_c])
response = Permute((2,1))(response)

In [ ]:
#Concatenate
answer = concatenate([response, question_encoded])

In [ ]:
answer

In [ ]:
answer = LSTM(32)(answer)

In [ ]:
answer = Dropout(0.5)(answer)
answer = Dense(vocab_len)(answer)

In [ ]:
answer = Activation('softmax')(answer)

In [ ]:
model = Model([input_sequence,question],answer)
# for a multi-class classification problem
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
#model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit([inputs_train, queries_train], answer_train,
                    batch_size = 32, epochs = 20,
                    validation_data = ([inputs_test, queries_test], answer_test)
                    )

In [ ]:
import matplotlib.pyplot as plt
print(history.history.keys())
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Model Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("epochs")
plt.legend(['train','test'], loc = 'upper right')
plt.show()

In [ ]:
#save
model.save("chatbot_model")

In [ ]:
#Evolution on the test set
model.load_weights("chatbot_model")

In [ ]:
pred_results = model.predict(([inputs_test,queries_test]))

In [ ]:
test_data[0][0]

In [ ]:
story = ' '.join(word for word in test_data[0][0])

In [ ]:
story

In [ ]:
query = ' '.join(word for word in test_data[0][1])

In [ ]:
query

In [ ]:
test_data[0][2]

In [ ]:
val_max = np.argmax(pred_results[0])

for key, value in tokenizer.word_index.items():
    if value==val_max:
        k = key

print("Predicted answer is ",k)
print("Probability of certainity ", pred_results[0][val_max])

In [ ]:
vocab

In [ ]:
my_story = "Mary dropped the football . Sandra is the in kitchen"

In [ ]:
my_story.split(" ")

In [ ]:
my_question = "Is Sandra in the kitchen ?"


In [ ]:
my_question.split(" ")

In [ ]:
my_answer = "yes"
my_answer.split(" ")


In [ ]:
my_data = [([my_story.split(), my_question.split(),'yes'])]
my_data

In [ ]:
my_sty, my_ques, my_ans = vectorize_stories(my_data)
#my_sty = (my_story.split())
#my_ques = my_question.split()
#my_ans = 'yes'
#my_sty = my_data[0][0]
#my_ques = my_data[0][1]
#my_ans = my_data[0][2]
#my_ans = vectorize_stories(my_data)
#my_sty = (my_story.split())
#my_ques = my_question.split()
#my_ans = 'yes'
print(my_ans)


In [ ]:

#pred_results = model.predict(([inputs_test,queries_test]))
pred_res = model.predict(([my_sty,my_ques]))
pred_res

In [ ]:
val_max = np.argmax(pred_res[0])

for key, value in tokenizer.word_index.items():
    if value==val_max:
        k = key

print("Predicted answer is ",k)
print("Probability of certainity ", pred_res[0][val_max])

In [ ]:
vocab.clear()
print(vocab)
for story, question, answer in my_data:
    vocab = vocab.union(set(story))
    vocab = vocab.union(set(question))
print(vocab)

In [ ]:
vocab.add('yes')
vocab

In [ ]:
val_max = np.argmax(pred_res[0])

for key, value in tokenizer.word_index.items():
    if value==val_max:
        k = key

print("Predicted answer is ",k)
print("Probability of certainity ", pred_res[0][val_max])

In [ ]:
tk = Tokenizer(filters= [])
tk.fit_on_texts(vocab)
print(tk.word_index)
print(tk.word_index.items())

In [ ]:
val_max = np.argmax(pred_res[0])
#print(val_max)
print(pred_res[0])
for key, value in tk.word_index.items():
    if value==val_max:
        k = key

print("Predicted answer is ",k)
print("Probability of certainity ", pred_res[0][val_max])